# Analysis of the number of teachers in German federal states

## 0. Introduction and Setup

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import src.data_lit_covid_school as dlcs
import matplotlib.pyplot as plt
from tueplots.constants.color import rgb



## 1. Data Import and Preparation

In [123]:
import warnings


def load_table(path: str = "../../data/destatis-summary.xlsx", sheet_name: str = "Tab. 7.1") -> pd.DataFrame:
    """Load table from destatis excel file"""
    df = pd.read_excel(os.path.join(dlcs.PACKAGE_PATH ,path), sheet_name=sheet_name, header=5, skipfooter=45, engine="openpyxl")
    zmw_col = df.columns[1]
    nested_ser = df[df[zmw_col].isna() == 1][df.columns[0]]
    school_ser = df[df[zmw_col] == "z"][df.columns[0]]
    
    # Preprocess such that all rows have the same values for the columns "School Type", "Contract Type" and "Federal State"
    
    # ---- School Type ----
    df["School Type"] = np.nan
    for index, value in school_ser.items():
        df.iloc[index, -1] = value
        df.iloc[index+1, -1] = value
        df.iloc[index+2, -1] = value
    
    # ---- Contract Type ----
    df["Contract Type"] = np.nan
    counter = 0
    new_nested_ser = nested_ser.copy()
    for index, value in nested_ser.items():
        if counter % 4 == 0:
            counter += 1
            continue
        
        times = nested_ser.index.tolist()[counter + 1] - index - 1 if counter + 1 < len(nested_ser) else len(df) - index - 1
        for i in range(times):
            df.iloc[index + 1 + i, -1] = value
        
        new_nested_ser = new_nested_ser.drop(index)
        counter += 1
    
    # ---- Federal State ----
    df["Federal State"] = np.nan
    counter = 0
    for index, value in new_nested_ser.items():
        times = new_nested_ser.index.tolist()[counter + 1] - index - 2 if counter + 1 < len(new_nested_ser) else len(df) - index - 2
        for i in range(times):
            df.iloc[index + 2 + i, -1] = value
        counter += 1
    
    # Preprocess the data into the melted format
    df = df[df["School Type"].isna() == 0]
    df = df[df.columns[-3:].tolist() + df.columns[:-3].tolist()]
    df = df.drop(columns=[df.columns[3]])
    df.rename(columns={df.columns[3]: "Gender"}, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.melt(id_vars=df.columns[:4].tolist(), value_vars=df.columns[4:].tolist(), var_name="Year", value_name="Number of Teachers")
    return df

# Load the data and display it
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    data = load_table()
data

,School Type,Contract Type,Federal State,Gender,Year,Number of Teachers
0,Schulkindergärten,Vollzeitbeschäftigte Lehrkräfte,Baden-Württemberg,z,1992,649.0
1,Schulkindergärten,Vollzeitbeschäftigte Lehrkräfte,Baden-Württemberg,m,1992,15.0
2,Schulkindergärten,Vollzeitbeschäftigte Lehrkräfte,Baden-Württemberg,w,1992,634.0
3,Grundschulen 1 2,Vollzeitbeschäftigte Lehrkräfte,Baden-Württemberg,z,1992,14539.0
4,Grundschulen 1 2,Vollzeitbeschäftigte Lehrkräfte,Baden-Württemberg,m,1992,8432.0
...,...,...,...,...,...,...
58256,Kollegs,Stundenweise beschäftigte Lehrkräfte,Deutschland,m,2020,87.0
58257,Kollegs,Stundenweise beschäftigte Lehrkräfte,Deutschland,w,2020,90.0
58258,Keine Zuordnung zu einer Schulart möglich 19,Stundenweise beschäftigte Lehrkräfte,Deutschland,z,2020,36.0
58259,Keine Zuordnung zu einer Schulart möglich 19,Stundenweise beschäftigte Lehrkräfte,Deutschland,m,2020,6.0


# Plot the number of teachers per federal state per year